## Instructions

You'll need to set environment variables for your secret keys: `OPENAI_API_KEY` and `ANTHROPIC_API_KEY`. 

In [20]:
!uv pip install llama-index \
    llama-index-embeddings-openai llama-index-llms-anthropic llama-index-llms-ollama  \
    llama-index-vector-stores-MongoDB \
    pymongo \
    vonage

Audited 7 packages in 22ms


In [21]:
# Check for some required environment variables:
import os

# Check all required environment variables have been set:
for required_env_var in ["OPENAI_API_KEY", "ANTHROPIC_API_KEY"]:
    try:
        assert os.environ[required_env_var] is not None
    except KeyError:
        print(f"You must set {required_env_var} to run this notebook!")

# Create an LLM object.

An LLM object provides an interface to send prompts to an LLM.
In this case, Anthropic credentials are silently provided by an env-var.

In [22]:
from llama_index.llms.anthropic import Anthropic

anthropic_llm = Anthropic(model="claude-3-5-sonnet-20241022")

# Create Some Maths Tools

This creates two tools, `multiply_tool` and `add_tool`.

In [ ]:
from llama_index.core.tools import FunctionTool

def multiply(a: int, b: int) -> int:
    """
    Multiply two numbers.
    
    Provide three parameters: a and b (the operands)."""
    return a * b

def add(a: int, b: int) -> int:
    """
    Add two numbers.
    
    Provide two parameters: a and b (the operands).
    """
    return a + b

multiply_tool = FunctionTool.from_defaults(fn=multiply)
add_tool = FunctionTool.from_defaults(fn=add)

# What's the difference between a tool and a function?
# Muppet reminder.

## Create an Agent

Create a ReActAgent with the provided tools.

In [ ]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(tools=[multiply_tool, add_tool], llm=anthropic_llm, verbose=True)
response = agent.chat("Solve ((5*4)+8)")
response.response

# Create a Tool to Send SMS

This tool will use the vonage API to send an SMS.
It's preconfigured to send messages to either my "wife" or my "manager".

(In fact all SMS messages will be routed to my phone.)

In [ ]:
import vonage

def send_sms(recipient: str, message: str):
    """
    Send an SMS to my manager or wife.
    
    The recipient argument should be either "manager" or "wife".
    The message argument should contain the message body.
    """
    destinations = {
        "manager": os.environ['DEST_PHONE_NUMBER'],
        "wife": os.environ['DEST_PHONE_NUMBER'],
    }

    nexmo_client = vonage.Client(key=os.environ['NEXMO_API_KEY'], secret=os.environ['NEXMO_API_SECRET'])
    nexmo_client.sms.send_message({
        "from": os.environ['FROM_PHONE_NUMBER'],
        "to": destinations[recipient],
        'text': message,
    })

send_sms_tool = FunctionTool.from_defaults(send_sms)

## Test Sending an SMS

Let's recreate that agent, and tell it to send an email.

In [ ]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools([multiply_tool, add_tool, send_sms_tool], llm=anthropic_llm, verbose=True)
agent.chat("Send an SMS to my boss telling him I'm going to be late for work.")

# It's 6:30pm and I'm still at work.
# It's 8:45am and I'm in the car.

# Where is the LLM getting its instructions?

We've seen that the LLM will take actions without being given instructions. Why?

In [ ]:
agent = ReActAgent.from_tools([multiply_tool, add_tool, send_sms_tool], llm=anthropic_llm, verbose=True)

for k, v in agent.get_prompts().items():
    print(f"Prompt: {k}\n\n{v.template}")

In [ ]:
import pymongo
from datetime import datetime, timedelta

mongodb_client = pymongo.MongoClient(os.environ['MONGODB_URI'])

class ActionLog:
    """
    Stores actions in MongoDB, and can retrieve all actions taken in the past day.
    """
    
    def __init__(self, mongodb_client):
        self._client = mongodb_client
        self.db = self._client.get_default_database()
        self.action_log = self.db.get_collection('action_log')
    
    def log_action(self, action_description: str):
        """ Record something you have done, so that it can be retrieved later.
        
        The action_description parameter should be a description of an individual action you have taken.
        """
        self.action_log.insert_one({
            'when': datetime.now(),
            'description': action_description,
        })

    def get_recent_actions(self):
        return list(self.action_log.find({
            'when': {"$gt": datetime.now() - timedelta(days=11)}
        }, {'_id': 0}).sort({'when': -1}))

action_log = ActionLog(mongodb_client)

log_action_tool = FunctionTool.from_defaults(action_log.log_action)
recent_actions_tool = FunctionTool.from_defaults(action_log.get_recent_actions)

action_log.get_recent_actions()

In [ ]:
agent = ReActAgent.from_tools([multiply_tool, add_tool, send_sms_tool, log_action_tool, recent_actions_tool], llm=anthropic_llm, verbose=True)

In [ ]:
response = agent.chat("It is 8:30am. Send an SMS to my boss to tell him I'm running late. Make up a plausible excuse.")
response

# Creating a store of facts

This will be a vector index storing everything I can think of that might be useful to the LLM in making decisions.
The LLM can use RAG techniques to look up anything it might need to know, and then can take action based on interpreting the results.

## Configure an Embedding Model

Vector search requires the data to be turned into an embedding, both when the data is loaded, and when the data is queried. 

In [ ]:
# Configure an embedding model
from llama_index.embeddings.openai import OpenAIEmbedding

embed_model = OpenAIEmbedding(
    model="text-embedding-3-small", 
    dimensions=256,
    embed_batch_size=10, 
)

## Create a Vector Index in MongoDB

This is slightly more complex than it could be, because vector indexes are created asynchronously, in a different service.

In [ ]:
# Create a vector index on the "facts" collection:
import time

from pymongo.collection import Collection
from pymongo.operations import SearchIndexModel

db = mongodb_client.get_default_database()

# If there isn't a "facts" collection, then create one:
if "facts" not in db.list_collection_names(filter={"name": "facts"}):   
    db.create_collection("facts")

# If there isn't a "facts_index" vector index, then create one:
facts_collection: Collection = db.get_collection("facts")
if not list(facts_collection.list_search_indexes(name="facts_index")):
    print("Creating vector index ...")
    facts_collection.create_search_index(model=SearchIndexModel({
        "fields": [
            {
            "numDimensions": 256,
            "path": "embedding",
            "similarity": "cosine",
            "type": "vector"
            }
        ]
        },
        name="facts_index",
        type="vectorSearch"))
    
    # Wait for it to be created:
    while True:
        indexes = list(facts_collection.list_search_indexes(name="facts_index"))
        if indexes and indexes[0].get('queryable'):
            break
        time.sleep(5)
    print("Done ...")


## Configure the Vector Store in LlamaIndex

* The `MongoDBAtlasVectorSearch` represents a collection of documents, and a vector index.
* The `VectorStoreIndex` is a combination of the store object, along with the embedding used to store data and query it.
* A query engine is built from the Index, and contains the parameters used to query the index, and interpret results (ie. it also knows what LLM to use to interpret results
* Finally, a `QueryEngineTool` is derived from the query engine and a description telling the LLM what the tool can be used to look up.

In [ ]:
# Configure MongoDB Vector Search
from llama_index.vector_stores.mongodb import MongoDBAtlasVectorSearch
from llama_index.core import VectorStoreIndex
from llama_index.core.tools import QueryEngineTool, ToolMetadata

vector_store = MongoDBAtlasVectorSearch(
    mongodb_client=mongodb_client,
    db_name=mongodb_client.get_default_database().name,
    collection_name="facts",
    vector_index_name="facts_index",
)

# Represents a store and an embedding:
index = VectorStoreIndex.from_vector_store(vector_store, embed_model=embed_model)

# Used to query the data:
query_engine = index.as_query_engine(similarity_top_k=5, llm=anthropic_llm)

# A tool! This is generated from the query_engine,
# combined with instructions to the LLM about what data can be looked up with it.
facts_tool = QueryEngineTool(query_engine=query_engine,
                             metadata=ToolMetadata(
        name="facts",
        description=(
            "Provides facts to help you make decisions."
            "Use a detailed plain text question as input to the tool."
        ),
    ),
)

## Load in some facts from text files

In [ ]:
# Load some facts:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
# from llama_index.storage.docstore.mongodb import MongoDocumentStore
from llama_index.core.schema import MetadataMode

reader = SimpleDirectoryReader(input_dir="./facts", required_exts=[".txt"])
documents = reader.load_data()

# create parser and parse document into nodes
parser = SentenceSplitter()
nodes = parser.get_nodes_from_documents(documents)

for node in nodes:
    node_embedding = embed_model.get_text_embedding(
        node.get_content(metadata_mode=MetadataMode.EMBED)
    )
    node.embedding = node_embedding

# build index
facts_collection.delete_many({})
vector_store.add(nodes)

# Test The Assistant!

In [ ]:
agent = ReActAgent.from_tools([multiply_tool, add_tool, send_sms_tool, log_action_tool, facts_tool], llm=anthropic_llm, verbose=True)

response = agent.chat("It is September 15th. Look up any relevant facts and act accordingly.")
response